# SenasJanAI Transformerモデル トレーニングハブ

## 1. 環境設定
必要なライブラリのインストールと、プロジェクトのセットアップを行います。

In [ ]:
# mahjongライブラリの特定バージョンを確実にインストール
!pip uninstall mahjong -y
!pip install mahjong==1.3.0

# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# プロジェクトディレクトリに移動
%cd "/content/drive/MyDrive/いろいろ/麻雀AI/googlecolab/senasJanAI"

## 2. データセットの準備
Google Driveから `data.zip` をコピーし、Colabの高速なローカルストレージに展開します。
**初回実行時、または `data.zip` の内容を更新した場合は必ず実行してください。**

In [ ]:
import os
import zipfile

ZIP_FILE_PATH = 'data.zip'
EXTRACT_DIR = '/content/data'

print(f"Copying '{ZIP_FILE_PATH}' to local environment...")
# ColabのローカルにZIPファイルをコピー (これにより展開が高速化する)
!cp -f "{ZIP_FILE_PATH}" /content/

print(f"Unzipping files to '{EXTRACT_DIR}'...")
# 既存の展開先フォルダがあれば削除して作り直す
if os.path.isdir(EXTRACT_DIR):
    !rm -rf {EXTRACT_DIR}
os.makedirs(EXTRACT_DIR, exist_ok=True)

with zipfile.ZipFile('/content/data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/') # /content/data/ に展開される

print("Dataset prepared successfully.")

## 3. データ生成
展開されたローカルの牌譜ログを解析し、モデルが学習できるベクトル形式のデータセットを生成します。
ローカルファイルを読むので、以前よりも**劇的に高速化**します。

In [ ]:
# 読み込み先がColabローカルの /content/data になるように、スクリプトを修正
!sed -i "s/DATA_DIR = 'data'/DATA_DIR = '\/content\/data'/g" src/transformer/generate_data.py

!python -m src.transformer.generate_data

## 4. モデルの学習
前のステップで生成したデータセットを読み込み、Transformerモデルの学習を開始します。

In [ ]:
!python -m src.transformer.train_transformer

## 5. 予測の実行
学習済みの最新モデルを使い、サンプルデータに対する予測を行います。

In [ ]:
!python -m src.transformer.predict_transformer